In [1]:
import numpy as np
import pulp as lp

In [2]:
print(lp.listSolvers(onlyAvailable=True))

['GLPK_CMD', 'CPLEX_PY']


In [3]:
# Tasks
tasks = np.array(range(1, 16))
# Agents
agents = np.array(range(1, 10))

# Cost Matrix
costs = np.array([
    [6,17,10,1,None,7,13,18,21,16,5,4,11,19,20],
    [9,None,21,4,5,None,11,8,0,32,7,3,24,0,0],
    [2,8,5,0,1,6,4,11,18,6,0,0,0,32,27],
    [19,31,19,20,9,11,15,2,None,None,None,None,None,5,21],
    [21,25,None,3,None,14,11,20,9,None,12,22,9,23,3],
    [11,5,21,None,18,20,21,None,17,12,7,11,1,2,1],
    [17,4,20,9,None,None,None,22,25,21,29,34,4,7,None],
    [4,None,3,2,4,7,9,1,None,12,33,16,11,None,12],
    [6,2,None,3,9,10,12,None,19,13,35,None,10,5,20]
])

print("tasks:", tasks, "Shape:", tasks.shape)
print("agents:", agents, "Shape:", agents.shape)
print("costs:", costs, "Shape:", costs.shape)

tasks: [ 1  2  3  4  5  6  7  8  9 10 11 12 13 14 15] Shape: (15,)
agents: [1 2 3 4 5 6 7 8 9] Shape: (9,)
costs: [[6 17 10 1 None 7 13 18 21 16 5 4 11 19 20]
 [9 None 21 4 5 None 11 8 0 32 7 3 24 0 0]
 [2 8 5 0 1 6 4 11 18 6 0 0 0 32 27]
 [19 31 19 20 9 11 15 2 None None None None None 5 21]
 [21 25 None 3 None 14 11 20 9 None 12 22 9 23 3]
 [11 5 21 None 18 20 21 None 17 12 7 11 1 2 1]
 [17 4 20 9 None None None 22 25 21 29 34 4 7 None]
 [4 None 3 2 4 7 9 1 None 12 33 16 11 None 12]
 [6 2 None 3 9 10 12 None 19 13 35 None 10 5 20]] Shape: (9, 15)


In [4]:
# Defining the linear programming problem
problem = lp.LpProblem(name="AssignmentProblem", sense=lp.LpMinimize)

In [5]:
# The cost data is made into a default dictionary
costs = lp.makeDict([agents, tasks], costs, 0)
print(type(costs))

<class 'collections.defaultdict'>


In [6]:
# Creates a list of tuples containing all the possible assignments
assignments = [(a, t) for a in agents for t in tasks]
print(assignments)

[(1, 1), (1, 2), (1, 3), (1, 4), (1, 5), (1, 6), (1, 7), (1, 8), (1, 9), (1, 10), (1, 11), (1, 12), (1, 13), (1, 14), (1, 15), (2, 1), (2, 2), (2, 3), (2, 4), (2, 5), (2, 6), (2, 7), (2, 8), (2, 9), (2, 10), (2, 11), (2, 12), (2, 13), (2, 14), (2, 15), (3, 1), (3, 2), (3, 3), (3, 4), (3, 5), (3, 6), (3, 7), (3, 8), (3, 9), (3, 10), (3, 11), (3, 12), (3, 13), (3, 14), (3, 15), (4, 1), (4, 2), (4, 3), (4, 4), (4, 5), (4, 6), (4, 7), (4, 8), (4, 9), (4, 10), (4, 11), (4, 12), (4, 13), (4, 14), (4, 15), (5, 1), (5, 2), (5, 3), (5, 4), (5, 5), (5, 6), (5, 7), (5, 8), (5, 9), (5, 10), (5, 11), (5, 12), (5, 13), (5, 14), (5, 15), (6, 1), (6, 2), (6, 3), (6, 4), (6, 5), (6, 6), (6, 7), (6, 8), (6, 9), (6, 10), (6, 11), (6, 12), (6, 13), (6, 14), (6, 15), (7, 1), (7, 2), (7, 3), (7, 4), (7, 5), (7, 6), (7, 7), (7, 8), (7, 9), (7, 10), (7, 11), (7, 12), (7, 13), (7, 14), (7, 15), (8, 1), (8, 2), (8, 3), (8, 4), (8, 5), (8, 6), (8, 7), (8, 8), (8, 9), (8, 10), (8, 11), (8, 12), (8, 13), (8, 14), 

In [7]:
# A dictionary called 'vars' is created to contain the decision variables
vars = lp.LpVariable.dicts(name="Assignment", indices=(agents, tasks), lowBound=0, upBound=None, cat=lp.LpBinary)

In [8]:
# The objective function is added to 'problem' first

problem += (
    lp.lpSum(
        [vars[a][t] * costs[a][t] for (a, t) in assignments if costs[a][t] is not None]
    ),
    "Sum_of_Assignment_Costs"
)

In [9]:
# There are column constraints. Each task can be assigned at most to one agent.
for t in tasks:
    problem += (lp.lpSum(vars[a][t] for a in agents) <= 1, f"task_{t}_constraint")

# There are row constraints. Each agent can be assigned to only one task.
for a in agents:
    problem += (lp.lpSum(vars[a][t] for t in tasks) == 1, f"agent_{a}_constraint")

# Infeasible assignments must be zero.
for (a,t) in assignments:
    if costs[a][t] is None:
        problem += (vars[a][t] == 0, f"assignment_({a},{t})_constraint")

In [10]:
print(problem)

AssignmentProblem:
MINIMIZE
6*Assignment_1_1 + 16*Assignment_1_10 + 5*Assignment_1_11 + 4*Assignment_1_12 + 11*Assignment_1_13 + 19*Assignment_1_14 + 20*Assignment_1_15 + 17*Assignment_1_2 + 10*Assignment_1_3 + 1*Assignment_1_4 + 7*Assignment_1_6 + 13*Assignment_1_7 + 18*Assignment_1_8 + 21*Assignment_1_9 + 9*Assignment_2_1 + 32*Assignment_2_10 + 7*Assignment_2_11 + 3*Assignment_2_12 + 24*Assignment_2_13 + 21*Assignment_2_3 + 4*Assignment_2_4 + 5*Assignment_2_5 + 11*Assignment_2_7 + 8*Assignment_2_8 + 2*Assignment_3_1 + 6*Assignment_3_10 + 32*Assignment_3_14 + 27*Assignment_3_15 + 8*Assignment_3_2 + 5*Assignment_3_3 + 1*Assignment_3_5 + 6*Assignment_3_6 + 4*Assignment_3_7 + 11*Assignment_3_8 + 18*Assignment_3_9 + 19*Assignment_4_1 + 5*Assignment_4_14 + 21*Assignment_4_15 + 31*Assignment_4_2 + 19*Assignment_4_3 + 20*Assignment_4_4 + 9*Assignment_4_5 + 11*Assignment_4_6 + 15*Assignment_4_7 + 2*Assignment_4_8 + 21*Assignment_5_1 + 12*Assignment_5_11 + 22*Assignment_5_12 + 9*Assignment_5_1

In [11]:
problem_cplex = problem.deepcopy()
problem_glpk = problem.deepcopy()

In [12]:
# The problem is solved using PuLP's CPLEX Solver
problem_cplex.solve(solver=lp.CPLEX_PY(msg=False))

# Print problem solver
print(f"\nSolver: {problem_cplex.solver}")

# Print the status of the problem
print(f"\nstatus: {problem_cplex.status}, {lp.LpStatus[problem_cplex.status]}")

# Print the assignments that were chosen, which agent-task pair
print("\nChosen agent-task pairs:")
for var in problem_cplex.variables():
    if var.value() == 1:
        print(f"{var.name} = {var.value()}")

# The optimal objective function value is printed to the screen
print(f"\nValue of Objective Function = {problem_cplex.objective.value()}")


# Print the value of slack variable for each constraint
print("\nValue of slack variables for each constraint:")
for name, constraint in problem_cplex.constraints.items():
    print(f"{name}: {constraint.value()}")


Solver: <pulp.apis.cplex_api.CPLEX_PY object at 0x00000237F2EB43D0>

status: 1, Optimal

Chosen agent-task pairs:
Assignment_1_4 = 1.0
Assignment_2_9 = 1.0
Assignment_3_11 = 1.0
Assignment_4_8 = 1.0
Assignment_5_15 = 1.0
Assignment_6_14 = 1.0
Assignment_7_13 = 1.0
Assignment_8_3 = 1.0
Assignment_9_2 = 1.0

Value of Objective Function = 17.0

Value of slack variables for each constraint:
task_1_constraint: -1.0
task_2_constraint: 0.0
task_3_constraint: 0.0
task_4_constraint: 0.0
task_5_constraint: -1.0
task_6_constraint: -1.0
task_7_constraint: -1.0
task_8_constraint: 0.0
task_9_constraint: 0.0
task_10_constraint: -1.0
task_11_constraint: 0.0
task_12_constraint: -1.0
task_13_constraint: 0.0
task_14_constraint: 0.0
task_15_constraint: 0.0
agent_1_constraint: 0.0
agent_2_constraint: 0.0
agent_3_constraint: 0.0
agent_4_constraint: 0.0
agent_5_constraint: 0.0
agent_6_constraint: 0.0
agent_7_constraint: 0.0
agent_8_constraint: 0.0
agent_9_constraint: 0.0
assignment_(1,5)_constraint: 0.0
ass

In [13]:
# The problem is solved using PuLP's GLPK Solver
problem_glpk.solve(solver=lp.GLPK(msg=False))

# Print problem solver
print(f"\nSolver: {problem_glpk.solver}")

# Print the status of the problem
print(f"\nstatus: {problem_glpk.status}, {lp.LpStatus[problem_glpk.status]}")

# Print the assignments that were chosen, which agent-task pair
print("\nChosen agent-task pairs:")
for var in problem_glpk.variables():
    if var.value() == 1:
        print(f"{var.name} = {var.value()}")

# The optimal objective function value is printed to the screen
print(f"\nValue of Objective Function = {problem_glpk.objective.value()}")


# Print the value of slack variable for each constraint
print("\nValue of slack variables for each constraint:")
for name, constraint in problem_glpk.constraints.items():
    print(f"{name}: {constraint.value()}")


Solver: <pulp.apis.glpk_api.GLPK_CMD object at 0x00000237F2EE74F0>

status: 1, Optimal

Chosen agent-task pairs:
Assignment_1_4 = 1
Assignment_2_9 = 1
Assignment_3_11 = 1
Assignment_4_8 = 1
Assignment_5_15 = 1
Assignment_6_14 = 1
Assignment_7_13 = 1
Assignment_8_3 = 1
Assignment_9_2 = 1

Value of Objective Function = 17

Value of slack variables for each constraint:
task_1_constraint: -1
task_2_constraint: 0
task_3_constraint: 0
task_4_constraint: 0
task_5_constraint: -1
task_6_constraint: -1
task_7_constraint: -1
task_8_constraint: 0
task_9_constraint: 0
task_10_constraint: -1
task_11_constraint: 0
task_12_constraint: -1
task_13_constraint: 0
task_14_constraint: 0
task_15_constraint: 0
agent_1_constraint: 0
agent_2_constraint: 0
agent_3_constraint: 0
agent_4_constraint: 0
agent_5_constraint: 0
agent_6_constraint: 0
agent_7_constraint: 0
agent_8_constraint: 0
agent_9_constraint: 0
assignment_(1,5)_constraint: 0
assignment_(2,2)_constraint: 0
assignment_(2,6)_constraint: 0
assignment_(